In [1]:
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F
from math import sqrt

/Users/baijiale/Documents/Code/recruitment_for_ml_jotang/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Tokenizer和输入处理

In [2]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "time flies like an arrow"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
print(inputs.input_ids)

tensor([[ 2051, 10029,  2066,  2019,  8612]])


* Tokenizer: AutoTokenizer基于BERT的bert-base-uncased模型，将输入文本分词并转化为ID向量。return_tensors="pt"指定返回的是PyTorch张量。
* 输入输出: 将文本 "time flies like an arrow" 转换成分词后的ID（词汇表中的整数索引）。输出类似tensor([[101, 2078, ...]])。

### 2. 嵌入层（Embedding Layer）

In [3]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
print(token_emb)

inputs_embeds = token_emb(inputs.input_ids)
print(inputs_embeds.size())

Embedding(30522, 768)
torch.Size([1, 5, 768])


* AutoConfig: AutoConfig从预训练模型中加载配置，获取vocab_size（词汇表大小）和hidden_size（隐藏层维度）。
* Embedding: nn.Embedding是PyTorch中的嵌入层，它把token ID映射到嵌入向量（大小为hidden_size）。
* inputs_embeds: 将输入的token ID转化为嵌入表示，形状为(batch_size, seq_len, hidden_size)。

在这段代码中，batch_size、seq_len 和 hidden_size 分别代表如下含义：

1.	batch_size:

* 含义: 批量大小，即同时处理的样本数量。
* 在自然语言处理中，通常我们会将多个句子组成一个批次，传入模型一次处理。
* 在你的代码中，inputs来自一个句子 "time flies like an arrow"，所以batch_size = 1（因为这里只有一个句子）。

2.	seq_len:

* 含义: 序列长度，即输入文本的token数量（分词后的长度）。
* 在你的示例中，输入句子 "time flies like an arrow"被分词为 [time, flies, like, an, arrow]，所以seq_len = 5。
* 如果是更长的句子或有多个句子，则seq_len会根据输入的不同而变化。

3.	hidden_size:

* 含义: 隐藏层的维度大小（即每个token的嵌入向量的维度）。BERT的hidden_size在bert-base-uncased模型中是 768。
* 这意味着每个输入token都被表示为一个768维的向量。

在代码中：

* inputs_embeds 的形状是 (batch_size, seq_len, hidden_size)，即 (1, 5, 768)。
* 1 表示有一个句子（batch_size）。
* 5 表示该句子中有5个token（seq_len）。
* 768 表示每个token用768维的向量表示（hidden_size）。

### 3. 自注意力计算

In [7]:
Q = K = V = inputs_embeds
dim_k = K.size(-1)
scores = torch.bmm(Q, K.transpose(1,2)) / sqrt(dim_k)
print(scores)

tensor([[[28.0974,  0.9781,  0.5916,  1.2912, -1.0577],
         [ 0.9781, 27.6146,  0.4287, -1.6370,  0.1412],
         [ 0.5916,  0.4287, 28.5915, -2.2223,  0.8335],
         [ 1.2912, -1.6370, -2.2223, 26.7196, -2.0823],
         [-1.0577,  0.1412,  0.8335, -2.0823, 24.6930]]],
       grad_fn=<DivBackward0>)


* Q, K, V: 在简单的自注意力机制中，Query（Q）、Key（K）和Value（V）通常是同一个输入（inputs_embeds）。
* dim_k: 代表Key的维度大小（通常与隐藏层维度一致），用于缩放点积注意力中的分母，防止数值过大。
* scores: 通过torch.bmm（批量矩阵乘法）计算Q和K的点积（*即将矩阵乘法对应位置的两个词向量（嵌入向量）做点乘*），再除以sqrt(dim_k)，得到注意力得分。scores的大小为(batch_size, seq_len, seq_len)，表示每个单词对其他单词的相关性。

在自注意力机制（Self-Attention）中，Query（Q）、Key（K）和Value（V）是输入数据的三种不同的表示，它们在计算注意力权重时扮演着不同的角色。具体来说，Q、K、V分别用来确定句子中每个词对于其他词的相关性和权重，最终生成注意力输出。以下是它们的详细解释：

1. Query（Q）

* 含义: Query可以看作是需要“查询”的词，它决定了我们要关注哪一个词的语义信息。
* 功能: Query与其他词的Key进行点积运算，得到当前词对其他词的相关性评分。简而言之，Query告诉我们”我要查询哪些信息”。
* 具体作用: 在自注意力机制中，对于每个词来说，它自己的嵌入向量会被映射成Query向量，用来与其他词的Key进行匹配。

2. Key（K）

* 含义: Key是每个词的表示，它和Query配对，用来决定该词在其他词的注意力分数中应占的比重。
* 功能: Query和Key的点积会告诉我们，这个Query和Key是否“相关”，或者说它们之间的语义距离有多近。
* 具体作用: Key可以理解为“标识符”，它表示该词的某种特征，允许其他词的Query和它进行比较，决定注意力分配。

3. Value（V）

* 含义: Value是每个词的表示，它存储了该词携带的所有信息。注意力权重将应用到Value上，生成新的表示。
* 功能: 在得到了Query和Key计算出的注意力权重后，Value会加权求和，最终输出该词对当前词的贡献信息。
* 具体作用: Value可以看作是携带实际信息的容器，经过加权后的Value向量将是自注意力层的输出。

在这段代码中，直接使用相同的嵌入向量来充当Query、Key和Value。在实际的自注意力机制中，Q、K、V通常是通过不同的线性变换矩阵生成的。

*可以看到，点积 scores 为一个 `(1 *) 5 * 5` 的对称矩阵，对角线上的项的值明显大得多，因为 token 与它自己是最相似的*

*如位置 ij 表示，第 i 个 token 与 第 j 个 token 的相似程度*

### 4. Softmax和权重计算

In [5]:
weights = F.softmax(scores, dim=-1)
print(weights.sum(dim=-1))

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)


* Softmax: 对注意力得分进行Softmax归一化，得到权重，表示每个单词对其他单词的注意力分布。
* weights.sum(dim=-1): 验证Softmax的归一化性质，即每一行的权重和应为1。

### 5. 注意力输出计算

In [6]:
attn_outputs = torch.bmm(weights, V)
print(attn_outputs.shape)

torch.Size([1, 5, 768])


attn_outputs: 使用注意力权重weights和Value V做加权求和，得到最终的注意力输出，大小为(batch_size, seq_len, hidden_size)。